<a href="https://colab.research.google.com/github/3veryDay/Project_Festival/blob/main/CAFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ 필수 라이브러리 설치

!pip install requests playwright
!playwright install

In [ ]:
import re
import os
import json
import uuid
import time
import asyncio
import requests
from playwright.async_api import async_playwright

# 네이버 디벨로퍼 애플리케이션 정보
client_id = "E1U56eaDwlyRGd0AZXyV"
client_secret = "4L7L8ni9aF"

def get_article_list_from_naver_api(name, max_results=1000):
    # API 요청 URL 및 헤더 설정 (카페/블로그에 따라 아래 url중 하나만 선택)
    url = "https://openapi.naver.com/v1/search/cafearticle.json" #카페
    # url = "https://openapi.naver.com/v1/search/blog.json" #블로그
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    # 축제명 쌍따옴표로 감싸서 검색 진행 -> 제목 또는 본문에 쌍따옴표로 감싸진 키워드가 들어간 게시물만 검색
    query = f'"{name}"'
    print(f"🔍 {query}로 검색 시작")

    results = []
    # 100개씩 최대 1000개 게시물 가져오기
    for start in range(1, max_results + 1, 100):
        params = {
            "query": query,
            "display": 100,
            "start": start
        }

        print(f"📦 {start}번부터 불러오는 중...")

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            items = data.get("items", [])
            results.extend(items)

            # 글이 더이상 없으면 불러오기 종료
            if len(items) < 100:
                print("더 이상 결과가 없습니다.")
                break
            time.sleep(1)  # 너무 빠른 요청 방지

    print(f"\n✅ 총 {len(results)}개 게시글 수집 완료!")
    return results


In [ ]:
async def get_article(num, url, name, page):
    try:
        await page.goto(url)
        await page.wait_for_timeout(1000)

        # iframe 접근 시도
        frame = page.frame(name="cafe_main")

        # 제목
        # 제목 요소가 일정 시간 안에 없으면 timeout 에러 발생 (가입이 필요한 카페 거르기용)
        await frame.wait_for_selector('h3.title_text', timeout=2500)
        title = await frame.locator('h3.title_text').inner_text()

        # 작성자
        writer = await frame.locator('button.nickname').first.inner_text()

        # 작성일
        date = await frame.locator('div.article_info span.date').inner_text()

        # 조회수
        view_raw = await frame.locator('div.article_info span.count').inner_text()
        view_count = int(re.sub(r'[^0-9]', '', view_raw))

        # 본문 텍스트
        paragraphs = await frame.locator('div.se-module-text p').all()
        content_texts = [await p.inner_text() for p in paragraphs]
        content = "\n".join(content_texts)

        # 댓글
        comments = []
        comment_elements = await frame.locator('ul.comment_list > li').all()
        for item in comment_elements:
            try:
                commenter = await item.locator('a.comment_nickname').inner_text()
                comment_text = await item.locator('span.text_comment').inner_text()
                comment_date = await item.locator('span.comment_info_date').inner_text()
                comments.append({
                    "작성자": commenter,
                    "내용": comment_text,
                    "날짜": comment_date
                })
            except:
                continue

        result = {
            'reviewID': str(uuid.uuid4()),
            'url': url,
            '글 제목': title,
            '글쓴이': writer,
            '축제명': name,
            '날짜': date,
            '조회수': view_count,
            '본문 글': content,
            '댓글': comments
        }

        print(f"✅ [{num + 1}] 크롤링 성공")
        print(result)
        return result

    except Exception as e:
        print(f"🚫 [{num + 1}] URL 접근 실패: {url}\n에러: {e}")
        return None

In [ ]:
def filter_relevant_posts(posts, keyword):
    # 공백 무시하는 정규표현식 패턴 생성
    pattern = r"".join([f"{char}\s*" for char in keyword])
    keyword_regex = re.compile(pattern, re.IGNORECASE)

    filtered = []
    for post in posts:
        title = re.sub(r"<.*?>", "", post["title"])
        desc = re.sub(r"<.*?>", "", post["description"])
        combined = title + " " + desc
        if re.search(keyword_regex, combined):
            filtered.append(post)
    print(f"✅ 필터링 후 남은 글 수: {len(filtered)}")
    return filtered

In [ ]:
async def main():
    # 크롤링할 축제명 입력
    name = "노원달빛산책"

    # 1. 게시글 리스트 받아오기
    raw_posts = get_article_list_from_naver_api(name)

    # 2.제목/요약에 정확히 축제명이 들어간 것만 필터링
    posts = filter_relevant_posts(raw_posts, name)

    # 3. 게시글 상세 내용 크롤링
    crawled_results = []
    async with async_playwright() as p:
        # 브라우저 설정
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # 크롤링 시작
        print("\n🔍 게시글 상세 내용 크롤링 시작...\n")
        for i, post in enumerate(posts):
            result = await get_article(i, post['link'], name, page)
            if result:
                crawled_results.append(result)

        await browser.close()

    # 3. JSON 파일로 저장 (축제명 기반 파일명)
    safe_name = name.replace(" ", "_")  # 공백을 밑줄로 바꾸기
    json_filename = f"results/{safe_name}.json"
    file_name = f"{safe_name}_카페_크롤링결과.json"
    path = '/content/drive/MyDrive/JSON/CAFE/' + file_name


    with open(path, 'w', encoding="utf-8") as f:
        json.dump(crawled_results, f, ensure_ascii=False, indent=2)
    print(f"\n📁 크롤링 결과 drive에 저장 완료: {json_filename}")

# 실행
await main()
